In [1]:
import pandas as pd
import numpy as np
import math

import time

In [2]:
# Importando .csv
train = pd.read_csv('dataset/train_data.csv')

In [3]:
def knn(train):
    ratings = train.pivot(index='user_id', columns='movie_id', values='rating')
    sim = ratings.corr(method='pearson')
    return { "sim": sim, "ratings": ratings}

In [4]:
# Implementando predição
def predict(model, u, i, k = 5):
    sim = model["sim"]
    ratings = model["ratings"]
    ratings_T = ratings.T
    if i not in sim or u not in ratings_T:
        return 0
    sim_items = sim[i][~sim[i].isna()].sort_values(ascending=False).index
    rated_items = ratings_T[u][ratings_T[u] > 0].index
    sim_k = np.intersect1d(sim_items, rated_items)
    top_k = [x for x in sim_items if x in sim_k][:k]
    sum_sim = 0
    dem = 0
    mean_u = ratings[i].mean(skipna = True)
    for v in top_k:
        dem += sim[i][v] * ratings[v][u] # - ratings[v].mean(skipna = True))
        sum_sim += sim[i][v]
    if sum_sim == 0:
        return 0
    return dem/sum_sim

In [5]:
# Avaliando modelo
def rmse(model, test, k = 5):
    sum_err = 0
    for t in test:
        u = t[0]
        i = t[1]
        r_ui = t[2]
        pred = predict(model, u, i, k)
        error = (r_ui - pred)**2
        sum_err += error
    return math.sqrt(sum_err/len(test))

In [6]:
def validation(model, data, train_split = 0.75, k = 5):
    data = data.sample(frac=1)
    train_len = int(train_split * len(data))
    train = data[:train_len]
    test = data[train_len:].values

    start_time = time.time()
    model = model(train)
    print("Tempo de treinamento em segundos: ", time.time() - start_time)
    start_time = time.time()
    print("RMSE :", rmse(model, test, k))
    print("Tempo de predição em segundos: ", time.time() - start_time)

In [7]:
# rmse(df.values[:1000])

In [8]:
validation(knn, train, 0.75, 9)

Tempo de treinamento em segundos:  188.4813015460968
RMSE : 1.3672508325733965
Tempo de predição em segundos:  1743.37872838974


## Execução 1 treino 75% k = 5
Tempo de treinamento em segundos:  147.43686604499817 <br />
RMSE : 1.1770065912219758 <br />
Tempo de predição em segundos:  1491.5968461036682 <br />

## Execução 1 treino 75% k = 9
Tempo de treinamento em segundos:  188.4813015460968 <br />
RMSE : 1.3672508325733965 <br />
Tempo de predição em segundos:  1743.37872838974 <br />